In [37]:
import dask
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import requests

In [38]:
df = pd.read_csv("DATASET/musiccaps-public.csv")
efficient_data_store = dict()  #key:caption, value:dict(start_s, end_s, ytid)

for index, row in df.iterrows():
    efficient_data_store[row['caption']] = {"start_s":row['start_s'], "end_s":row['end_s'], "ytid":row['ytid']}

captions = np.array(df.caption)

In [39]:
QUERY = "I want a slow and mellow song for sleep"

In [40]:
VDB_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
query_embedding = VDB_model.encode(QUERY)
passage_embedding = VDB_model.encode(captions)
similarity = util.dot_score(query_embedding, passage_embedding)
sorted_similarity = similarity.sort(descending=True)

In [41]:
print("Caption that matched your query: ", captions[int(sorted_similarity[1][0][0])])

Caption that matched your query:  This is a song that would be suitable as meditation music or background music for sleeping. It is calming and features sustained high pitched notes on a pad synth. The song feels ethereal and relaxing.


In [42]:
music_clip = efficient_data_store[captions[int(sorted_similarity[1][0][0])]]
URL = f"https://youtu.be/watch?v={music_clip['ytid']}&start={music_clip['start_s']}"
print(f"YouTube link to that music clip: {URL}")

YouTube link to that music clip: https://youtu.be/watch?v=aNJEh3_Vmps&start=30


In [ ]:
API_URL_R = "https://api-inference.huggingface.co/models/bert-large-uncased-whole-word-masking-finetuned-squad"
headers_R = {"Authorization": }


def query_RoBERTa(payload):
	response = requests.post(API_URL_R, headers=headers_R, json=payload)
	return response.json()

def get_top_k_similar_paragraphs(Paragraphs, sorted_similarity_score, top_k_val=3):
    return [int(sorted_similarity_score[1][0][i]) for i in range(0,top_k_val)], [Paragraphs[int(sorted_similarity_score[1][0][i])] for i in range(0,top_k_val)]


In [ ]:
indices, Top_K_contexts = get_top_k_similar_paragraphs(lyrics, sorted_similarity, 10)
for i in range(len(Top_K_contexts)): print(f"k={i+1}) {Top_K_contexts[i]}")
print(indices)